<a href="https://colab.research.google.com/github/saerarawas/AAI614_rawas/blob/main/Week%203/Notebook3_2_Rawas_Saera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AAI614: Data Science & its Applications

*Notebook 3.2: Practice with Data Cleaning*

<a href="https://colab.research.google.com/github/harmanani/AAI614/blob/main/Week%203/Notebook3.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

Exercise I. Load the following datafile from GitHub

In [2]:
grads = pd.read_csv("https://raw.githubusercontent.com/harmanani/AAI614/main/Week%203/grads.csv")

In [3]:
grads

,Student Name,Avg Hours Studies per Week,GPA,University,Sense of Humour (0-5),Salary
0,George,20,NaN,NYU,3.0,$40k
1,Jerry,35,3.5,Columbia,5.0,$80k
2,Elaine,55,4.0,Columbia,4.2,$60k
3,Cosmo,5,2.0,City College,2.0,$25k
4,Newman,25,2.8,City College,0.0,$50k
5,Frank,35,3.0,Festivus Uni,NaN,$40k
6,Estelle,100,3.2,Festivus Uni,1.7,$0k
7,Leo,15,2.4,Festivus Uni,0.0,$35k
8,Rachel,50,4.0,Columbia,NaN,$75k


In [6]:
grads.head()

,Student Name,Avg Hours Studies per Week,GPA,University,Sense of Humour (0-5),Salary
0,George,20,NaN,NYU,3.0,$40k
1,Jerry,35,3.5,Columbia,5.0,$80k
2,Elaine,55,4.0,Columbia,4.2,$60k
3,Cosmo,5,2.0,City College,2.0,$25k
4,Newman,25,2.8,City College,0.0,$50k


Question 1: Identify all the outliers in the above data.  Justify your answers using objective measures.

In [8]:
from scipy import stats
import numpy as np

# Select only numeric columns for z-score calculation
numeric_grads = grads.select_dtypes(include=np.number)
z_scores = np.abs(stats.zscore(numeric_grads))
outliers = grads[(z_scores > 3).any(axis=1)]
print(outliers)

# Select only numeric columns for IQR calculation
Q1 = numeric_grads.quantile(0.25)
Q3 = numeric_grads.quantile(0.75)
IQR = Q3 - Q1
outliers_iqr = grads[((numeric_grads < (Q1 - 1.5 * IQR)) | (numeric_grads > (Q3 + 1.5 * IQR))).any(axis=1)]
print(outliers_iqr)


Empty DataFrame
Columns: [Student Name, Avg Hours Studies per Week, GPA, University, Sense of Humour (0-5), Salary]
Index: []
  Student Name  Avg Hours Studies per Week  GPA     University  \
6      Estelle                         100  3.2  Festivus Uni    

   Sense of Humour (0-5) Salary  
6                    1.7    $0k  


Question 2: There are various data that are missing.  Fill-in the missing data or delete the rows/columns that you think you should delete.  Justify your answer

In [9]:
print(grads.isnull().sum())

Student Name                  0
Avg Hours Studies per Week    0
GPA                           1
University                    0
Sense of Humour (0-5)         2
Salary                        0
dtype: int64


In [11]:
grads_dropped_rows = grads.dropna()

In [14]:
threshold = 0.5 * len(grads)  # Example threshold: delete columns with more than 50% missing values
grads_dropped_columns = grads.dropna(thresh=threshold, axis=1)
numeric_grads = grads.select_dtypes(include=np.number)
grads_mean_imputed = grads.fillna(numeric_grads.mean())
grads_mode_imputed = grads.apply(lambda x: x.fillna(x.mode()[0]) if x.dtype == 'object' else x)
grads_ffill = grads.fillna(method='ffill')
grads_bfill = grads.fillna(method='bfill')
grads_interpolated = grads.interpolate()


<ipython-input-14-8531a054a7f8>:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grads_ffill = grads.fillna(method='ffill')
<ipython-input-14-8531a054a7f8>:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grads_bfill = grads.fillna(method='bfill')
<ipython-input-14-8531a054a7f8>:8: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  grads_interpolated = grads.interpolate()


Question 3: Reload the data and fill-in the data using mean method as well as the frequent method.

In [15]:
import pandas as pd
import ssl

# Create an unverified SSL context to fetch the data
ssl._create_default_https_context = ssl._create_unverified_context

# Load the dataset
url = "https://raw.githubusercontent.com/harmanani/AAI614/main/Week%203/grads.csv"
grads = pd.read_csv(url)

# Fill missing values using the mean for numerical columns
grads_mean_imputed = grads.apply(lambda col: col.fillna(col.mean()) if col.dtype in ['float64', 'int64'] else col)

# Fill missing values using the mode for categorical columns
grads_frequent_imputed = grads_mean_imputed.apply(lambda col: col.fillna(col.mode()[0]) if col.dtype == 'object' else col)

# Display the dataframe with imputed values
print(grads_frequent_imputed)


  Student Name  Avg Hours Studies per Week     GPA     University  \
0       George                          20  3.1125            NYU   
1        Jerry                          35  3.5000       Columbia   
2       Elaine                          55  4.0000       Columbia   
3        Cosmo                           5  2.0000   City College   
4       Newman                          25  2.8000   City College   
5        Frank                          35  3.0000   Festivus Uni   
6      Estelle                         100  3.2000  Festivus Uni    
7          Leo                          15  2.4000   Festivus Uni   
8       Rachel                          50  4.0000       Columbia   

   Sense of Humour (0-5) Salary  
0               3.000000   $40k  
1               5.000000   $80k  
2               4.200000   $60k  
3               2.000000   $25k  
4               0.000000   $50k  
5               2.271429   $40k  
6               1.700000    $0k  
7               0.000000   $35k  
8  

Exercise II. Run the cell below to create a new dataframe called `df_miss`.  Its first column will contain some missing values.

In [16]:
import pandas as pd
import numpy as np
import random

nrows = 10
ncols = 5

# set a seed for random number generation
np.random.seed(314)
# create an array filled with random data
data = np.array(np.random.rand(nrows, ncols))
# put the data to a pandas dataframe
df_miss = pd.DataFrame(data)
# rename the columns
df_miss.columns = ['col_'+str(ii) for ii in range(ncols)]

# randomly set some values to missing
ix0 = np.random.randint(nrows, size=3)
ix1 = np.random.randint(nrows, size=3)

df_miss['col_0'][ix0] = np.nan
df_miss['col_1'][ix1] = np.nan

print(df_miss)

      col_0     col_1     col_2     col_3     col_4
0       NaN       NaN  0.265048  0.783205  0.918001
1  0.827355       NaN  0.260480  0.911763  0.260757
2  0.766376  0.261531  0.122291  0.386006  0.840081
3       NaN       NaN  0.633110  0.584766  0.581232
4  0.677205  0.687155  0.438927  0.320927  0.570552
5       NaN  0.861074  0.834805  0.105766  0.060408
6  0.596882  0.792395  0.226356  0.535201  0.136066
7  0.372244  0.151977  0.429822  0.792706  0.406957
8  0.177850  0.909252  0.545331  0.100497  0.718721
9  0.978429  0.309776  0.260126  0.662900  0.139720


<ipython-input-16-a77e6ba6a8cb>:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_miss['col_0'][ix0] = np.nan
<ipython-input-16-a77e6ba6a8cb>:22: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting 

Impute the missing values (NaN) in `col_0` (but not `col_1`) with the median.  Store the values in the dataframe by using the parameter `inplace`.  Print the dataframe.

In [17]:
# Impute missing values in col_0 with the median
df_miss['col_0'].fillna(df_miss['col_0'].median(), inplace=True)

# Print the DataFrame after imputation
print("\nDataFrame after Imputing Missing Values in col_0 with the Median:")
print(df_miss)



DataFrame after Imputing Missing Values in col_0 with the Median:
      col_0     col_1     col_2     col_3     col_4
0  0.677205       NaN  0.265048  0.783205  0.918001
1  0.827355       NaN  0.260480  0.911763  0.260757
2  0.766376  0.261531  0.122291  0.386006  0.840081
3  0.677205       NaN  0.633110  0.584766  0.581232
4  0.677205  0.687155  0.438927  0.320927  0.570552
5  0.677205  0.861074  0.834805  0.105766  0.060408
6  0.596882  0.792395  0.226356  0.535201  0.136066
7  0.372244  0.151977  0.429822  0.792706  0.406957
8  0.177850  0.909252  0.545331  0.100497  0.718721
9  0.978429  0.309776  0.260126  0.662900  0.139720


<ipython-input-17-d6ae9efb70d5>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_miss['col_0'].fillna(df_miss['col_0'].median(), inplace=True)


Impute the missing values in `col_1` with value 0.  Store the values in the dataframe by using the parameter `inplace`.  Print the dataframe.

In [18]:
# Impute missing values in col_1 with 0
df_miss['col_1'].fillna(0, inplace=True)

# Print the DataFrame after imputation
print("\nDataFrame after Imputing Missing Values in col_1 with 0:")
print(df_miss)



DataFrame after Imputing Missing Values in col_1 with 0:
      col_0     col_1     col_2     col_3     col_4
0  0.677205  0.000000  0.265048  0.783205  0.918001
1  0.827355  0.000000  0.260480  0.911763  0.260757
2  0.766376  0.261531  0.122291  0.386006  0.840081
3  0.677205  0.000000  0.633110  0.584766  0.581232
4  0.677205  0.687155  0.438927  0.320927  0.570552
5  0.677205  0.861074  0.834805  0.105766  0.060408
6  0.596882  0.792395  0.226356  0.535201  0.136066
7  0.372244  0.151977  0.429822  0.792706  0.406957
8  0.177850  0.909252  0.545331  0.100497  0.718721
9  0.978429  0.309776  0.260126  0.662900  0.139720


<ipython-input-18-9c80ad449c5f>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_miss['col_1'].fillna(0, inplace=True)
